In [79]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [80]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [81]:
def compute_accuracy(v_xs , v_ys ):
    global prediction
    y_pre = sess.run(prediction , feed_dict = {xs:v_xs , keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre ,1 ) , tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction ,tf.float32))
    result = sess.run(accuracy , feed_dict = {xs : v_xs , ys :v_ys , keep_prob:1})
    return result

In [82]:
def weight_varialbe(shape):
    initial = tf.truncated_normal(shape , stddev=0.1)
    return tf.Variable(initial)

In [83]:
def bias_variable(shape):
    initial = tf.constant(0.1 , shape = shape)
    return tf.Variable(initial)

In [84]:
def con2D( x, w):
    return tf.nn.conv2d( x, w , strides=[1,1,1,1] , padding='SAME')

In [85]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1] , strides=[1,2,2,1] , padding='SAME')

In [86]:
xs = tf.placeholder(tf.float32 , [None , 784])/255
ys = tf.placeholder(tf.float32 , [None , 10])
keep_prob = tf.placeholder(tf.float32)

In [87]:
x_image = tf.reshape( xs , [-1 , 28 , 28 ,1])

In [88]:
print(x_image)

Tensor("Reshape_6:0", shape=(?, 28, 28, 1), dtype=float32)


In [89]:
W_conv1 = weight_varialbe([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(con2D(x_image ,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [92]:
W_conv2 = weight_varialbe([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(con2D(h_pool1 ,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [95]:
W_fc1 = weight_varialbe([7*7*64, 1024])
b_fc1 = bias_variable([1024])

In [96]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [98]:
W_fc2 = weight_varialbe([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [99]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))       # loss
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()

In [100]:
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)


In [101]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
    if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images[:1000], mnist.test.labels[:1000])
             )

0.123
0.752
0.86
0.894
0.905
0.924
0.917
0.933
0.944
0.941
0.946
0.947
0.958
0.951
0.952
0.964
0.965
0.964
0.966
0.971
